In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class DiscreteGradientOperator(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        self.shape = shape

    def _matvec(self, x):
        x = x.reshape(self.shape)
        gradient_x = np.zeros(self.shape)
        gradient_x[:, :-1] = np.diff(x, axis=1)
        gradient_y = np.zeros(self.shape)
        gradient_y[:-1, :] = np.diff(x, axis=0)
        return np.vstack((gradient_x, gradient_y)).ravel()

    def _rmatvec(self, x):
        x = x.reshape((2, self.shape[0], self.shape[1]))
        adjoint_x = np.zeros(self.shape)
        adjoint_x[:, :-1] = -np.diff(x[0], axis=1)
        adjoint_y = np.zeros(self.shape)
        adjoint_y[:-1, :] = -np.diff(x[1], axis=0)
        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 3, 3
operator = DiscreteGradientOperator((M, N))

# Testing with a random input
input_vector = np.random.rand(M, N)
output_vector = operator.matvec(input_vector.ravel())
adjoint_output_vector = operator.rmatvec(output_vector)

# Reshape results for visualization
output_matrix = output_vector.reshape((2, M, N))
adjoint_output_matrix = adjoint_output_vector.reshape((M, N))

print("Input matrix:")
print(input_vector)

print("\nOutput matrix (Gradient):")
print(output_matrix)

print("\nAdjoint Output matrix:")
print(adjoint_output_matrix)


ValueError: dimension mismatch

In [4]:
input_vector.ravel().shape

(9,)

# Working

In [21]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class DiscreteGradientOperator(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        # self.shape = shape
        self._shape = shape

    def _matvec(self, x):
        x = x.reshape(self._shape)
        gradient_x = np.zeros(self._shape)
        gradient_x[:, :-1] = np.diff(x, axis=1)
        gradient_y = np.zeros(self._shape)
        gradient_y[:-1, :] = np.diff(x, axis=0)
        return np.vstack((gradient_x, gradient_y)).ravel()

    def _rmatvec(self, x):
        x = x.reshape((2, self._shape[0], self._shape[1]))
        adjoint_x = np.zeros(self._shape)
        adjoint_x[:, :-1] = -np.diff(x[0], axis=1)
        adjoint_y = np.zeros(self._shape)
        adjoint_y[:-1, :] = -np.diff(x[1], axis=0)
        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 3, 3
operator = DiscreteGradientOperator((M, N))

# Testing with a random input
input_vector = np.random.rand( M * N)
output_vector = operator.matvec(input_vector)
adjoint_output_vector = operator.rmatvec(output_vector)

# Reshape results for visualization
output_matrix = output_vector.reshape((2, M, N))
adjoint_output_matrix = adjoint_output_vector.reshape((M, N))

print("Input matrix:")
print(input_vector)

print("\nOutput matrix (Gradient):")
print(output_matrix)

print("\nAdjoint Output matrix:")
print(adjoint_output_matrix)


Input matrix:
[0.24268599 0.69366231 0.50293159 0.37640175 0.0072602  0.0696412
 0.76010158 0.78677962 0.72341919]

Output matrix (Gradient):
[[[ 0.45097631 -0.19073072  0.        ]
  [-0.36914155  0.06238101  0.        ]
  [ 0.02667803 -0.06336043  0.        ]]

 [[ 0.13371576 -0.68640211 -0.43329038]
  [ 0.38369984  0.77951942  0.65377799]
  [ 0.          0.          0.        ]]]

Adjoint Output matrix:
[[ 0.39172296 -1.65665225 -1.08706837]
 [-0.04782272  0.84190043  0.65377799]
 [ 0.09003846 -0.06336043  0.        ]]


In [25]:

operator = DiscreteGradientOperator((M, N))

# Validation using random vectors
x = np.random.rand(M * N)
y = np.random.rand(2 * M * N)

# Calculate <Ax, y>
Ax_y = np.inner(operator.matvec(x), y)

# Calculate <x, A^*y>
x_Aty = np.inner(x, operator.rmatvec(y))

# Check if the two inner products are approximately equal
tolerance = 1e-10
if np.abs(Ax_y - x_Aty) < tolerance:
    print("Adjoint test passed!")
else:
    print("Adjoint test failed.")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")

print("\nInput matrix:")
print(input_vector)

print("\nOutput matrix (Gradient):")
print(output_matrix)

print("\nAdjoint Output matrix:")
print(adjoint_output_matrix)


Adjoint test failed.
<Ax, y> = -0.09034321750935831, <x, A^*y> = -0.8466242826500311

Input matrix:
[0.24268599 0.69366231 0.50293159 0.37640175 0.0072602  0.0696412
 0.76010158 0.78677962 0.72341919]

Output matrix (Gradient):
[[[ 0.45097631 -0.19073072  0.        ]
  [-0.36914155  0.06238101  0.        ]
  [ 0.02667803 -0.06336043  0.        ]]

 [[ 0.13371576 -0.68640211 -0.43329038]
  [ 0.38369984  0.77951942  0.65377799]
  [ 0.          0.          0.        ]]]

Adjoint Output matrix:
[[ 0.39172296 -1.65665225 -1.08706837]
 [-0.04782272  0.84190043  0.65377799]
 [ 0.09003846 -0.06336043  0.        ]]


### Adjoint test

In [22]:
for j in range(10):
    x = np.random.normal(size=(M,N)).flatten()
    y = np.random.normal(size=(2,M,N)).flatten()
    Rx = operator.matvec(x)
    Rty = operator.rmatvec(y)
    dot1 = (Rx*y).sum()
    dot2 = (x*Rty).sum()
    print(f"dot1 = {dot1}")
    print(f"dot2 = {dot2}")
    print(f"")

dot1 = 4.198691211077149
dot2 = -5.4347773802340145

dot1 = -0.29210075312691375
dot2 = 3.7159732466348467

dot1 = -6.054453569215181
dot2 = 13.11307263217163

dot1 = -0.6963332292809347
dot2 = 0.041842216145886124

dot1 = -3.0442870286015884
dot2 = -0.28854972899138565

dot1 = -2.2796566826809235
dot2 = -2.93777954659334

dot1 = 2.134062879232609
dot2 = -1.460348447066913

dot1 = -0.20834094584167206
dot2 = 0.274262440507433

dot1 = -9.4686826892827
dot2 = 3.176296683808627

dot1 = 3.102252710154131
dot2 = 3.4899976361411085



# With Neumann?

In [30]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class DiscreteGradientOperatorNeumann(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        self._shape = shape

    def _matvec(self, x):
        x = x.reshape(self._shape)
        gradient_x = np.zeros(self._shape)
        gradient_x[:, :-1] = np.diff(x, axis=1)
        gradient_x[:, -1] = x[:, 0] - x[:, -1] # Neumann boundary condition in x
        gradient_y = np.zeros(self._shape)
        gradient_y[:-1, :] = np.diff(x, axis=0)
        gradient_y[-1, :] = x[0, :] - x[-1, :] # Neumann boundary condition in y
        return np.vstack((gradient_x, gradient_y)).ravel()

    def _rmatvec(self, x):
        x = x.reshape((2, self._shape[0], self._shape[1]))
        adjoint_x = np.zeros(self._shape)
        adjoint_x[:, 0] = x[0, :, -1] - x[0, :, 0] # Neumann boundary condition in x
        adjoint_x[:, 1:] = -np.diff(x[0], axis=1)
        adjoint_y = np.zeros(self._shape)
        adjoint_y[0, :] = x[1, -1, :] - x[1, 0, :] # Neumann boundary condition in y
        adjoint_y[1:, :] = -np.diff(x[1], axis=0)
        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 3, 3
operator = DiscreteGradientOperatorNeumann((M, N))

# Validation using random vectors
x = np.random.rand(M * N)
y = np.random.rand(2 * M * N)

# Calculate <Ax, y>
Ax_y = np.inner(operator.matvec(x), y)

# Calculate <x, A^*y>
x_Aty = np.inner(x, operator.rmatvec(y))

# Check if the two inner products are approximately equal
tolerance = 1e-10
if np.abs(Ax_y - x_Aty) < tolerance:
    print("Adjoint test passed!")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")
else:
    print("Adjoint test failed.")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")


Adjoint test passed!
<Ax, y> = -0.2523205463114665, <x, A^*y> = -0.2523205463114666


In [20]:
input_vector.shape

(9,)

In [17]:
operator.shape

(18, 9)

In [18]:
operator.matvec(input_vector)

ValueError: cannot reshape array of size 9 into shape (18,9)

# Dirichlet?

In [32]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class DiscreteGradientOperatorDirichlet(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        self._shape = shape

    def _matvec(self, x):
        x = x.reshape(self._shape)
        gradient_x = np.zeros(self._shape)
        gradient_x[:, :-1] = np.diff(x, axis=1)
        gradient_y = np.zeros(self._shape)
        gradient_y[:-1, :] = np.diff(x, axis=0)
        return np.vstack((gradient_x, gradient_y)).ravel()

    def _rmatvec(self, x):
        x = x.reshape((2, self._shape[0], self._shape[1]))
        adjoint_x = np.zeros(self._shape)
        adjoint_x[:, 1:] = -np.diff(x[0], axis=1)
        adjoint_y = np.zeros(self._shape)
        adjoint_y[1:, :] = -np.diff(x[1], axis=0)
        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 3, 3
operator = DiscreteGradientOperatorDirichlet((M, N))

# Validation using random vectors
x = np.random.rand(M * N)
y = np.random.rand(2 * M * N)

# Calculate <Ax, y>
Ax_y = np.inner(operator.matvec(x), y)

# Calculate <x, A^*y>
x_Aty = np.inner(x, operator.rmatvec(y))

# Check if the two inner products are approximately equal
tolerance = 1e-10
if np.abs(Ax_y - x_Aty) < tolerance:
    print("Adjoint test passed!")
else:
    print("Adjoint test failed.")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")


Adjoint test failed.
<Ax, y> = 0.08396794531985324, <x, A^*y> = -1.8475726079954633


In [35]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class DiscreteGradientOperatorDirichlet(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        self._shape = shape

    def _matvec(self, x):
        x = x.reshape(self._shape)
        gradient_x = np.diff(x, axis=1)
        gradient_y = np.diff(x, axis=0)
        
        # Apply Dirichlet zero boundary condition
        gradient_x = np.hstack((gradient_x, np.zeros((self._shape[0], 1))))
        gradient_y = np.vstack((gradient_y, np.zeros((1, self._shape[1]))))
        
        return np.vstack((gradient_x, gradient_y)).ravel()

    def _rmatvec(self, x):
        x = x.reshape((2, self._shape[0], self._shape[1]))
        adjoint_x = -np.diff(x[0], axis=1)
        adjoint_y = -np.diff(x[1], axis=0)
        
        # Remove last column and last row to account for Dirichlet zero boundary condition
        adjoint_x = adjoint_x[:, :-1]
        adjoint_y = adjoint_y[:-1, :]
        
        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 3, 3
operator = DiscreteGradientOperatorDirichlet((M, N))

# Validation using random vectors
x = np.random.rand(M * N)
y = np.random.rand(2 * M * N)

# Calculate <Ax, y>
Ax_y = np.inner(operator.matvec(x), y)

# Calculate <x, A^*y>
x_Aty = np.inner(x, operator.rmatvec(y))

# Check if the two inner products are approximately equal
tolerance = 1e-10
if np.abs(Ax_y - x_Aty) < tolerance:
    print("Adjoint test passed!")
else:
    print("Adjoint test failed.")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")


Adjoint test failed.
<Ax, y> = -0.34561712332317446, <x, A^*y> = -1.2217100556005915


In [38]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class DiscreteGradientOperatorDirichlet(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        self._shape = shape
        self.M, self.N = shape

    def _matvec(self, x):
        x = x.reshape(self._shape)
        gradient_x = np.zeros((self.M, self.N))
        gradient_y = np.zeros((self.M, self.N))

        gradient_x[:, :-1] = np.diff(x[:self.M,:], axis=1)
        gradient_y[:-1, :] = np.diff(x[self.M:,:], axis=0)

        return np.vstack((gradient_x.ravel(), gradient_y.ravel()))

    def _rmatvec(self, x):
        x = x.reshape((2, self.M, self.N))

        adjoint_x = np.zeros((self.M, self.N))
        adjoint_y = np.zeros((self.M, self.N))

        adjoint_x[:, 0] = -x[0, :, 0]
        adjoint_x[:, -1] = x[0, :, -1]
        adjoint_x[:, 1:-1] = -np.diff(x[0], axis=1)

        adjoint_y[0, :] = -x[1, 0, :]
        adjoint_y[-1, :] = x[1, -1, :]
        adjoint_y[1:-1, :] = -np.diff(x[1], axis=0)

        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 3, 3
operator = DiscreteGradientOperatorDirichlet((M, N))

# Validation using random vectors
x = np.random.rand(M * N)
y = np.random.rand(2* M * N)

# Calculate <Ax, y>
Ax_y = np.inner(operator.matvec(x), y)

# Calculate <x, A^*y>
x_Aty = np.inner(x, operator.rmatvec(y))

# Check if the two inner products are approximately equal
tolerance = 1e-10
if np.abs(Ax_y - x_Aty) < tolerance:
    print("Adjoint test passed!")
else:
    print("Adjoint test failed.")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")


ValueError: could not broadcast input array from shape (0,3) into shape (2,3)